In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kapaa,US,2021-09-06 19:23:07,22.0752,-159.3190,82.72,69,20,3.00
1,1,Punta Arenas,CL,2021-09-06 19:22:10,-53.1500,-70.9167,53.71,62,90,11.50
2,2,Blagoveshchensk,RU,2021-09-06 19:23:08,50.2667,127.5333,61.41,100,40,0.00
3,3,Port Elizabeth,ZA,2021-09-06 19:21:45,-33.9180,25.5701,55.09,94,75,6.91
4,4,Vaini,TO,2021-09-06 19:23:09,-21.2000,-175.2000,64.56,94,20,1.14


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kapaa,US,2021-09-06 19:23:07,22.0752,-159.3190,82.72,69,20,3.00
8,8,Sinnamary,GF,2021-09-06 19:19:55,5.3833,-52.9500,82.44,76,100,9.15
24,24,Omboue,GA,2021-09-06 19:23:17,-1.5746,9.2618,75.45,82,10,8.32
27,27,Atuona,PF,2021-09-06 19:23:18,-9.8000,-139.0333,78.60,70,44,19.84
28,28,Puerto Ayora,EC,2021-09-06 19:23:18,-0.7393,-90.3518,78.76,78,100,10.13
29,29,Manokwari,ID,2021-09-06 19:23:19,-0.8667,134.0833,77.45,83,69,2.73
30,30,Castro,BR,2021-09-06 19:19:19,-24.7911,-50.0119,83.28,24,0,1.92
32,32,Sao Filipe,CV,2021-09-06 19:23:20,14.8961,-24.4956,80.55,79,44,6.42
37,37,Butaritari,KI,2021-09-06 19:23:22,3.0707,172.7902,83.46,72,100,13.09
39,39,Ranong,TH,2021-09-06 19:23:23,9.9658,98.6348,78.91,87,95,6.44


In [9]:
preferred_cities_df.count()

City_ID       192
City          192
Country       190
Date          192
Lat           192
Lng           192
Max Temp      192
Humidity      192
Cloudiness    192
Wind Speed    192
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,82.72,22.0752,-159.3190,
8,Sinnamary,GF,82.44,5.3833,-52.9500,
24,Omboue,GA,75.45,-1.5746,9.2618,
27,Atuona,PF,78.60,-9.8000,-139.0333,
28,Puerto Ayora,EC,78.76,-0.7393,-90.3518,
29,Manokwari,ID,77.45,-0.8667,134.0833,
30,Castro,BR,83.28,-24.7911,-50.0119,
32,Sao Filipe,CV,80.55,14.8961,-24.4956,
37,Butaritari,KI,83.46,3.0707,172.7902,
39,Ranong,TH,78.91,9.9658,98.6348,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,82.72,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Sinnamary,GF,82.44,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
24,Omboue,GA,75.45,-1.5746,9.2618,Hotel Olako
27,Atuona,PF,78.60,-9.8000,-139.0333,Villa Enata
28,Puerto Ayora,EC,78.76,-0.7393,-90.3518,Finch Bay Galapagos Hotel
29,Manokwari,ID,77.45,-0.8667,134.0833,Swiss Belhotel Manokwari
30,Castro,BR,83.28,-24.7911,-50.0119,CHACARA BAILLY
32,Sao Filipe,CV,80.55,14.8961,-24.4956,Hotel Xaguate
37,Butaritari,KI,83.46,3.0707,172.7902,Isles Sunset Lodge
39,Ranong,TH,78.91,9.9658,98.6348,Tinidee Hotel@Ranong


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [20]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))